# Introduction a PyTorch

based on https://github.com/mila-udem/welcome_tutorials/tree/master/pytorch By Sandeep Subramanian

https://github.com/jcjohnson/pytorch-examples

In [1]:
import torch
import numpy as np
from torch.autograd import Variable

torch.from_numpy(np.array([1,2,3,4]))


 1
 2
 3
 4
[torch.LongTensor of size 4]

In [2]:
torch.LongTensor([1,2,3,4])
torch.Tensor([1,2,3,4]).type(torch.LongTensor)


 1
 2
 3
 4
[torch.LongTensor of size 4]

In [3]:
x = torch.Tensor([1,2,3,4])

In [4]:
x.sqrt()


 1.0000
 1.4142
 1.7321
 2.0000
[torch.FloatTensor of size 4]

In [5]:
x = x.view(-1, 1)

In [6]:
torch.cat([x, x], 1)


 1  1
 2  2
 3  3
 4  4
[torch.FloatTensor of size 4x2]

## Autograd

In [7]:
x = Variable(x, requires_grad=False)
w = Variable(torch.ones(4, 1), requires_grad=True)
w

Variable containing:
 1
 1
 1
 1
[torch.FloatTensor of size 4x1]

In [8]:
print(w.grad)

None


In [9]:
z = torch.dot(w, x)
print(z)

Variable containing:
 10
[torch.FloatTensor of size 1]



## Optim and loss function

In [10]:
optimizer = torch.optim.SGD([w], lr=0.01)

In [11]:
optimizer.zero_grad()
z.backward()
print(w.grad)
optimizer.step()

Variable containing:
 1
 2
 3
 4
[torch.FloatTensor of size 4x1]



In [12]:
print(w)

Variable containing:
 0.9900
 0.9800
 0.9700
 0.9600
[torch.FloatTensor of size 4x1]



In [13]:
loss_fn = torch.nn.MSELoss()
z = torch.dot(w, x)
target = Variable(torch.zeros(1))
optimizer.zero_grad()
loss = loss_fn(z, target)
loss.backward()
optimizer.step()
print(w)

Variable containing:
 0.7960
 0.5920
 0.3880
 0.1840
[torch.FloatTensor of size 4x1]



# Fashion MNIST

based on MNIST tutorial: https://github.com/pytorch/examples/blob/master/mnist/main.py

In [14]:
%matplotlib inline  
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torchvision
import torch.nn.functional as F
from fashion import FashionMNIST
import torchvision.transforms as transforms
from torch import nn
from torch import optim

In [15]:
train_data = FashionMNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

valid_data = FashionMNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [16]:
train_idx = np.random.choice(train_data.train_data.shape[0], 54000, replace=False)

In [17]:
train_data.train_data = train_data.train_data[train_idx, :]
train_data.train_labels = train_data.train_labels[torch.from_numpy(train_idx).type(torch.LongTensor)]

KeyboardInterrupt: 

In [ ]:
mask = np.ones(60000)
mask[train_idx] = 0

In [ ]:
valid_data.train_data = valid_data.train_data[torch.from_numpy(np.argwhere(mask)), :].squeeze()
valid_data.train_labels = valid_data.train_labels[torch.from_numpy(mask).type(torch.ByteTensor)]

In [ ]:
batch_size = 100
test_batch_size = 100

train_loader = torch.utils.data.DataLoader(train_data,
    batch_size=batch_size, shuffle=True)

valid_loader = torch.utils.data.DataLoader(valid_data,
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    FashionMNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)


In [ ]:
plt.imshow(train_loader.dataset.train_data[1].numpy())

In [ ]:
plt.imshow(train_loader.dataset.train_data[10].numpy())

In [ ]:
class FcNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, image):
        batch_size = image.size()[0]
        x = image.view(batch_size, -1)
        x = F.sigmoid(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=0)
        return x

In [ ]:
model = FcNetwork()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(loader, name):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(loader.dataset)
    print('\n' + name + ' set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

epochs = 5
for epoch in range(1, epochs + 1):
    train(epoch)
    test(valid_loader, 'valid')

test(test_loader, 'test')
